In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

def compute_calibration_para(images, nx, ny):
    # prepare object points,like (0,0,0), (1,0,0), (2,0,0) ....,(nx,ny,0)
    objp = np.zeros((ny * nx, 3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
    
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    return objpoints, imgpoints

In [ ]:
def cal_undistort(img, objpoints, imgpoints):
    img_size = img.shape[0:2]
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    return undist

images = glob.glob('camera_cal/calibration*.jpg')
objpoints, imgpoints = compute_calibration_para(images, 9, 6)
img = cv2.imread('camera_cal/calibration2.jpg')
img_undist = cal_undistort(img, objpoints, imgpoints)
cv2.imwrite('output_images/undist_calibration2.jpg', img_undist)

In [ ]:
def threshold_bin(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    
    return color_binary

In [ ]:
def warper(img, src, dst):
    img_size = (img.shape[1], img.shape[0])
    # Compute and apply perpective transform
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)  # keep same size as input image
    
    return warped, M

In [ ]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty

In [ ]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    ### Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    return left_fit, right_fit, left_fitx, right_fitx, ploty

In [ ]:
def search_around_poly(left_fit, right_fit, binary_warped):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fit, right_fit, left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    return left_fit, right_fit, left_fitx, right_fitx, ploty

In [ ]:
def visualize_lane(binary_warped, leftx, lefty, rightx, righty):
    margin = 100
    # Fit new polynomials
    left_fit, right_fit, left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)

    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    return result

In [ ]:
def draw_lane_lines(img, left_fitx, right_fitx, ploty):
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(img, np.int_([pts]), (0,255, 0))
    
    return img

def draw_text(img, text, org = (50, 50)):
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1
    color = (255, 255, 255)
    thickness = 2
    img_txt = cv2.putText(img, text, org, font,  
                   fontScale, color, thickness, cv2.LINE_AA)

    return img_txt

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):

    return cv2.addWeighted(initial_img, α, img, β, γ)

In [ ]:
def write_pipeline_image(img_name, img):
    cv2.imwrite('output_images/' + img_name, img)

In [ ]:
class Lane:
    def __init__(self):
        self.calibrationDone = False
        self.objpoints = None
        self.imgpoints = None
        self.usePrevFrame = False
        self.left_fit = None
        self.right_fit = None

def pipeline(img, lane=Lane(), pipe_img_name=None):
    nx = 9
    ny = 6

    x_size = img.shape[1]
    y_size = img.shape[0]

    # 1. Calibration
    if lane.calibrationDone == False:
        lane.calibrationDone = True
        print('camera calibration')
        # Make a list of calibration images
        images = glob.glob('camera_cal/calibration*.jpg')
        lane.objpoints, lane.imgpoints = compute_calibration_para(images, nx, ny)

    # 2. Distortion correction
    img_undist = cal_undistort(img, lane.objpoints, lane.imgpoints)
    if pipe_img_name: write_pipeline_image('undist_' + pipe_img_name, img_undist)

    # 3. Threshold binary
    img_bin = threshold_bin(img_undist)
    if pipe_img_name: write_pipeline_image('bin_' + pipe_img_name, img_bin)

    # 4. Perspective transform
    offset_x_top = 55
    offset_x_bottom = x_size / 4 - offset_x_top
    offset_y = 100

    src = np.float32([[(x_size / 2 - offset_x_top), (y_size / 2 + offset_y)],
                      [(x_size / 2 + offset_x_top), (y_size / 2 + offset_y)],
                      [(x_size - offset_x_bottom), y_size],
                      [offset_x_bottom, y_size]])

    offset_x = x_size / 4
    dst = np.float32([[offset_x, 0],
                      [x_size - offset_x, 0],
                      [x_size - offset_x, y_size],
                      [offset_x, y_size]])

    img_wrp, M = warper(img_bin, src, dst)
    if pipe_img_name: write_pipeline_image('wrp_' + pipe_img_name, img_wrp)

    # 5. Detecting lane pixels
    img_gry = cv2.cvtColor(img_wrp, cv2.COLOR_RGB2GRAY).astype(np.uint8)

    leftx, lefty, rightx, righty = find_lane_pixels(img_gry)
    
    if lane.usePrevFrame == False:
        print('polyfit')
        lane.left_fit = np.polyfit(lefty, leftx, 2)
        lane.right_fit = np.polyfit(righty, rightx, 2)

    lane.left_fit, lane.right_fit, left_fitx, right_fitx, ploty = search_around_poly(lane.left_fit, lane.right_fit, img_gry)
    if pipe_img_name:
        img_vis = visualize_lane(img_gry, leftx, lefty, rightx, righty)
        write_pipeline_image('vis_' + pipe_img_name, img_vis)

    # 6. Measuring curvature
    left_curverad, right_curverad = measure_curvature_real(leftx, lefty, rightx, righty)
    
    # 7. Drawing lane lines
    img_lane = draw_lane_lines(img_wrp, left_fitx, right_fitx, ploty)

    img_unwrp, M = warper(img_lane, dst, src)
    if pipe_img_name: write_pipeline_image('unwrp_' + pipe_img_name, img_unwrp)

    img_wg = weighted_img(img_unwrp, img, 1, 0.3, 0)
    
    # 8. Writing text info
    text = 'Radius of Curvature = ' + str(int(left_curverad)) + '(m)'
    dist, dirx = measure_dist_center(x_size, y_size, lane.left_fit, lane.right_fit)
    img_out = draw_text(img_wg, text)
    text = 'Vehicle is ' + str(round(dist, 2)) + 'm ' + dirx + ' of center'
    img_out = draw_text(img_out, text, (50,100))
    if pipe_img_name: write_pipeline_image('out_' + pipe_img_name, img_out)

    print(left_curverad, right_curverad, dist, dirx)

    return img_out, lane

In [ ]:
image = cv2.imread('test_images/straight_lines2.jpg')
img_out, _ = pipeline(image)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(img_out)
ax2.set_title('Output Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
import os
images = os.listdir("test_images/")

for img_name in images:
    img = cv2.imread('test_images/' + img_name)
    pipeline(img, pipe_img_name=img_name)

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

lane = Lane()

def process_image(image):
    global lane
    img_out, lane = pipeline(image, lane)
    lane.usePrevFrame = True
    return img_out

video_output = 'project_video_out.mp4'

# clip2 = VideoFileClip('project_video.mp4').subclip(0,5)
clip2 = VideoFileClip('project_video.mp4')
video_clip = clip2.fl_image(process_image)
%time video_clip.write_videofile(video_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))